In [13]:
#IMPORTS E MAIS IMPORTS
import numpy as np 
import pandas as pd
import os
import PIL
import random
from PIL import Image
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
#USEI OS PROPRIOS NOMES DAS PASTAS DO DATASET COMO LABELS DE TREINAMENTO 
directory = "C:/Users/essec/Desktop/fufs2/PokemonData"
labels = os.listdir(directory)
nb = len(labels)

In [5]:
#DAQUI PRA BAIXO È SÒ PARA TRANSFORMAR AS IMAGENS EM ARRAY, REDIMENSIONA-LAS E NORNMALIZA-LAS (SÓ PARTE CHATA....)
stored = {}
def input_target_split(train_dir,labels):
    dataset = []
    count = 0
    for label in labels:
        folder = os.path.join(train_dir,label)
        for image in os.listdir(folder):
            try:
                img=load_img(os.path.join(folder,image), target_size=(150,150))
                img=img_to_array(img)
                img=img/255.0
                dataset.append((img,count))
            except:
                pass

        print(f'\rCompleted: {label}',end='')
        stored[label] = count
        count+=1
    random.shuffle(dataset)
    X, y = zip(*dataset)
    
    return np.array(X),np.array(y)

In [6]:
X, y = input_target_split(directory,labels)

Completed: Zubatstuffdslash

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.22, random_state=42)

In [8]:
datagen = ImageDataGenerator(horizontal_flip=True,
                             vertical_flip=True,
                             rotation_range=20,
                             zoom_range=0.2,
                             width_shift_range = 0.2,
                             height_shift_range = 0.2,
                             shear_range=0.1,
                             fill_mode="nearest")

testgen = ImageDataGenerator()

datagen.fit(X_train)
testgen.fit(X_test)

In [9]:
y_train = np.eye(nb)[y_train]
y_test = np.eye(nb)[y_test]

In [11]:
#USANDO DENSENET201 PARA DEIXAR MAIS RAPIDO, COM MENOS ERROS E MAIS PRATICO
#LIMITANDO A DENSENET A 675 LAYERS ._.
from tensorflow.keras.applications import DenseNet201

img_size = 150
base_model = DenseNet201(include_top = False,
                         weights = 'imagenet',
                         input_shape = (img_size,img_size,3))

for layer in base_model.layers[:675]:
    layer.trainable = False

for layer in base_model.layers[675:]:
    layer.trainable = True

In [12]:
#MODELINHO SEQUENCIAL PADRÃO  
model = Sequential()

model.add(base_model)
#GLOBALAVERAGE PQ COMBINA ACHEI ELA BEM EFICIENTE PARA POKEMONS
model.add(GlobalAveragePooling2D())
model.add(Dense(nb, activation=tf.nn.softmax))
                                             #OTIMIZADOR ADAM PQ È O MAIS COMUM                                      PRECISAO 
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), loss = 'categorical_crossentropy', metrics=['accuracy'])


In [ ]:
#CAMINHO DO SAVE
path = "C:/Users/essec/Desktop/"
                            #SALVAMOS O MODELO TREINADO EM .H5
model.save(os.path.join(path,"fufs2/modelsave.h5"))


In [ ]:
#TREINAR ESSE MODELO USANDO QUANTOS EPOCHS SEU COMPUTADOR AGUENTAR :)
EPOCHS = 50

hist = model.fit(datagen.flow(X_train,y_train,batch_size=32),
                                        validation_data=testgen.flow(X_test,y_test,batch_size=32),
                                        epochs=EPOCHS)


In [22]:
#BASICAMENTE PARA TESTAR O MODELO (ISSO TA NO BOT)
import cv2
image = cv2.imread('C:/Users/essec/Desktop/bulb3.jpg')
img = cv2.resize(image, (150, 150))
img=img/255.0
img = np.expand_dims(img, axis=0)
pred = new_model.predict(img)
label = np.argmax(pred,axis=1)
print(labels[label[0]])

Rattata
